In [ ]:
# 분리된 카페 리뷰를 하나의 카페로 병합

def review_sum():
    
    import pandas as pd
    tokenized = pd.read_csv('../data/final (2).csv')
    
    # (1) 데이터 형태 변경
    import ast
    tokenized['contents'] = tokenized['contents'].apply(lambda x: ast.literal_eval(x))
    tokenized['noun'] = tokenized['noun'].apply(lambda x: ast.literal_eval(x))
    tokenized['adj'] = tokenized['adj'].apply(lambda x: ast.literal_eval(x))
    
    # (2) Word2Vector 적용을 위한 토큰 추출
    tokenized['verb'] = tokenized['contents'].apply(lambda x: [w[0] for w in x if w[1]=='Verb'])
    
    
    # (3) 전체 카페 리스트 dataframe 생성
    cafe_unique = tokenized[['name', 'category', 'location', 'rating', 'rating_num', 'review_num', 'link', 'Americano']].drop_duplicates(keep='first')
    
    # (4) 카페별 리뷰에서 추출된 명사 토큰 합치기
    for i in cafe_unique.index:
        link = cafe_unique.loc[i,'link']
        review_to_sum = tokenized[tokenized['link']==link].reset_index(drop=True)
        result = []
        for j in review_to_sum.index:
            result += review_to_sum.loc[j,'noun']
        cafe_unique.loc[i,'noun'] = str(result)
        
    # (5) 카페별 리뷰에서 추출된 형용사 토큰 합치기
    for i in cafe_unique.index:
        link = cafe_unique.loc[i,'link']
        review_to_sum = tokenized[tokenized['link']==link].reset_index(drop=True)
        result = []
        for j in review_to_sum.index:
            result += review_to_sum.loc[j,'adj']
        cafe_unique.loc[i,'adj'] = str(result)
    
    # (6) 카페별 리뷰에서 추출된 동사 토큰 합치기
    for i in cafe_unique.index:
        link = cafe_unique.loc[i,'link']
        review_to_sum = tokenized[tokenized['link']==link].reset_index(drop=True)
        result = []
        for j in review_to_sum.index:
            result += review_to_sum.loc[j,'verb']
        cafe_unique.loc[i,'verb'] = str(result)
    
    # (7) 데이터 형태 변경
    cafe_unique['noun'] = cafe_unique['noun'].apply(lambda x: ast.literal_eval(x))
    cafe_unique['adj'] = cafe_unique['adj'].apply(lambda x: ast.literal_eval(x))
    cafe_unique['verb'] = cafe_unique['verb'].apply(lambda x: ast.literal_eval(x))
    
    # (8) 카페별 전체 토큰 합치기
    cafe_unique['token'] = cafe_unique['noun'] + cafe_unique['adj'] + cafe_unique['verb']
    
    # (9) 카페별 리뷰 dataframe 저장
    cafe_unique.to_csv('../data/word2vec_final.csv', index=False)
    
    return cafe_unique

In [ ]:
# 문장 벡터 생성 (문장을 구성하는 토큰 벡터의 평균)

def get_sentence_mean_vector(token):
    vector = []
    for i in token:
        try:
            vector.append(w2v_total.wv[i])
        except KeyError as e:
            pass
    try:
        return np.mean(vector, axis=0)
    except IndexError as e:
        pass

cafe_unique['wv_total'] = cafe_unique['token'].map(get_sentence_mean_vector)

In [ ]:
# 카페별 벡터 생성 (앞서 정의된 get_sentence_mean_vector 사용)

def cafe_to_vector(cafe_unique):
    
    # (1) Word2Vec을 통해 추출된 토큰의 벡터(150차원) 생성
    from gensim.models import Word2Vec
    tokens = list(cafe_unique['token'].values)
    w2v_total = Word2Vec(sentences=tokens, vector_size=150, window=3, min_count=1, workers=4, sg=1)
    
    # (2) 카페별 벡터 생성
    cafe_unique['wv_total'] = cafe_unique['token'].map(get_sentence_mean_vector)
    
    return cafe_unique

In [ ]:
# 카페 유형 분류

def cafe_clustering(cafe_unique):
    
    from sklearn.cluster import KMeans
    
    word_vectors = cafe_unique.wv_total_noun.to_list() 
    num_clusters = 4

    kmeans_clustering = KMeans(n_clusters = num_clusters, random_state=np.random.RandomState(seed=1))
    kmeans_clustering.fit(word_vectors)
    idx = kmeans_clustering.fit_predict( word_vectors )
    cafe_unique['k=4'] = idx
    
    return cafe_unique

In [ ]:
# 카페별 카테고리 점수 산정

def cafe_score(cafe_unique, num_clusters):
    
    # (1) 카페 클러스터별 중심점 

    # 감성카페(cluster 0) center point
    cluster_0 = kmeans_clustering.cluster_centers_[0]

    # 디저트카페 (cluster 3) center point
    cluster_1 = kmeans_clustering.cluster_centers_[1]

    # 프랜차이즈 카페 center point
    cluster_2 = kmeans_clustering.cluster_centers_[2]

    # 카공 카페 center point
    cluster_3 = kmeans_clustering.cluster_centers_[3]
    
    # (2) 중심과의 거리 계산
    
    from scipy.spatial import distance
    cafe_unique['감성_dist'] = cafe_unique['wv_total_noun'].apply(lambda x: distance.euclidean(x,cluster_0))
    cafe_unique['디저트_dist'] = cafe_unique['wv_total_noun'].apply(lambda x: distance.euclidean(x,cluster_1))
    cafe_unique['프랜차이즈_dist'] = cafe_unique['wv_total_noun'].apply(lambda x: distance.euclidean(x,cluster_2))
    cafe_unique['공부_dist'] = cafe_unique['wv_total_noun'].apply(lambda x: distance.euclidean(x,cluster_3))
    
    # (3) 점수화를 위한 거리 scaling
    
    from sklearn.preprocessing import MinMaxScaler

    # 감성카페
    gamsung_scaler = MinMaxScaler()
    cafe_unique['감성_scaled'] = gamsung_scaler.fit_transform(cafe_unique[['감성_dist']])
    
    # 디저트카페
    dessert_scaler = MinMaxScaler()
    cafe_unique['디저트_scaled'] = dessert_scaler.fit_transform(cafe_unique[['디저트_dist']])
    
    # 프랜차이즈카페
    franchise_scaler = MinMaxScaler()
    cafe_unique['프랜차이즈_scaled'] = franchise_scaler.fit_transform(cafe_unique[['프랜차이즈_dist']])
    
    # 카공 카페
    study_scaler = MinMaxScaler()
    cafe_unique['공부_scaled'] = study_scaler.fit_transform(cafe_unique[['study_dist']])
    
    # (4) 최종 점수 계산 : (1-거리)의 제곱
    cafe_unique['감성_score'] = cafe_unique['감성_scaled'].apply(lambda x: (1-x)**2)
    cafe_unique['디저트_score'] = cafe_unique['dessert_scaled'].apply(lambda x: (1-x)**2)
    cafe_unique['프랜차이즈_score'] = cafe_unique['franchise_scaled'].apply(lambda x: (1-x)**2)
    cafe_unique['공부_score'] = cafe_unique['study_scaled'].apply(lambda x: (1-x)**2)
    
    cafe_unique.to_csv('../data/cluster_result.csv')
    
    return cafe_unique

In [ ]:
# 지역별 카페 유형 점수 산정

def location_cafe_type_score(cafe_unique):
    
    # (1) 카페별 지역 정보 추출
    cafe_unique['si'] = cafe_unique['location'].apply(lambda x: x.split(' ')[0])
    cafe_unique['gu'] = cafe_unique['location'].apply(lambda x: x.split(' ')[1])
    
    # (2) 지역별 카페 유형 점수 산정
    location_score = cafe_unique.groupby('gu').agg({'gamsung_score':'mean','gamsung_score':'mean','dessert_score':'mean','franchise_score':'mean', 'study_score':'mean'}).reset_index()
    location_score.to_csv('../data/지역별 크롤링 비율')
    
    return location_score

In [ ]:
cafe_unique = review_sum()
cafe_unique = cafe_to_vector(cafe_unique)
cafe_clustering = cafe_clustering(cafe_unique)
cafe_unique = cafe_score(cafe_unique)
location_score = location_cafe_type_score(cafe_unique)